<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UncertaintyBootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import clone_model

import PIL
from PIL import Image

import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras import optimizers
from keras.callbacks import *
from sklearn.metrics import *
from keras.models import load_model
import tensorflow_addons as tfa

from torchvision import transforms

import tensorflow as tf
import tensorflow.keras.backend as backend
import math
import gc

# SVHN

In [ ]:
# from scipy.io import loadmat
# train_raw = loadmat('train_32x32.mat')
# test_raw = loadmat('test_32x32.mat')

In [ ]:
# def dummy_labels(labels):
#   zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
#   for i in range(labels.shape[0]):
#     zero_labels[i][labels[i]] = 1
#   return(zero_labels)

In [ ]:
# train_images = train_raw['X']
# train_labels = train_raw['y']

# test_images = test_raw['X']
# test_labels = dummy_labels(test_raw['y']-1)

# train_images = train_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)
# test_images = test_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)

In [ ]:
# temp = [0,0,0,0,0,0,0,0,0,0]
# label_indx = []
# unlabel_indx = []

# for i in range(73257) :
#   if temp[(train_labels).reshape([-1])[i]-1] < 25 :
#     temp[(train_labels).reshape([-1])[i]-1] += 1
#     label_indx.append(i)
#   else :
#     unlabel_indx.append(i)

In [ ]:
# lbl_train_images = train_images[label_indx]
# lbl_train_labels = dummy_labels(train_labels[label_indx]-1)

In [ ]:
# ubl_train_images = train_images[unlabel_indx]
# ubl_train_labels = dummy_labels(train_labels[unlabel_indx]-1)

# CIFAR 10

In [ ]:
cifar10 = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images
test_images = test_images

In [ ]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [ ]:
train_labels = dummy_labels(train_labels)
test_labels = dummy_labels(test_labels)

In [ ]:
# 1000 labeled, 49000 unlabeled
random.seed(10)
indx = random.sample(range(train_labels.shape[0]),train_labels.shape[0])

lbl_train_images = train_images[indx[:1000]]
ubl_train_images = train_images[indx[1000:]]

lbl_train_labels = train_labels[indx[:1000]]
ubl_train_labels = train_labels[indx[1000:]]

# valids1 =  train_images[indx[800:1000]]
# valids2 =  train_labels[indx[800:1000]]

#MAin


In [ ]:
class SGDR(Callback):

    def __init__(self, min_lr=0.0, max_lr=0.03, base_epochs=20, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        tide = ((self.cycles == 0) * 1) * (self.cycle_iterations*self.max_lr + (self.base_epochs - self.cycle_iterations)*self.min_lr) / self.base_epochs + ((self.cycles != 0) * 1)*(self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs)))
        return tide
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_13():
  conv1a = Conv2D(128, (3,3), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(128, (3,3), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(128, (3,3), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = PermaDropout(0.5)

  conv2a = Conv2D(256, (3,3), padding = 'same')
  bn2a = BatchNormalization()
  conv2b = Conv2D(256, (3,3), padding = 'same')
  bn2b = BatchNormalization()
  conv2c = Conv2D(256, (3,3), padding = 'same')
  bn2c = BatchNormalization()
  pl2 = MaxPooling2D(2, 2)
  MCdrop2 = PermaDropout(0.5)

  conv3a = Conv2D(512, (3,3))
  bn3a = BatchNormalization()
  conv3b = Conv2D(256, (1,1))
  bn3b = BatchNormalization()
  conv3c = Conv2D(128, (1,1))
  bn3c = BatchNormalization()
  pl3 = AveragePooling2D(6, 2)

  fc = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                      keras.Input(shape=(32, 32, 3)), 
                      tfa.layers.WeightNormalization(conv1a), bn1a, activ,
                      tfa.layers.WeightNormalization(conv1b), bn1b, activ,
                      tfa.layers.WeightNormalization(conv1c), bn1c, activ,
                      pl1, MCdrop1,

                      tfa.layers.WeightNormalization(conv2a), bn2a, activ,
                      tfa.layers.WeightNormalization(conv2b), bn2b, activ,
                      tfa.layers.WeightNormalization(conv2c), bn2c, activ,
                      pl2, MCdrop2,

                      tfa.layers.WeightNormalization(conv3a), bn3a, activ,
                      tfa.layers.WeightNormalization(conv3b), bn3b, activ,
                      tfa.layers.WeightNormalization(conv3c), bn3c, activ,
                      pl3, Flatten(),
                      
                      fc
                      ])
  
  return model

def compile_cnn_13(model):

  opt = keras.optimizers.SGD(0.03, momentum=0.9)

  model.compile(
    optimizer = opt,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

  return model

def cnn_13():

  model = create_cnn_13()
  model = compile_cnn_13(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):

  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
  sgdr = SGDR(min_lr=0.0, max_lr=0.03, base_epochs=20) #스케줄러

  model.fit(
      x=X,
      y=y,
      epochs=Epoch,
      verbose=0,
      batch_size=Batch,
      callbacks=[sgdr]
  )
    
  model_test_eval(model, test_images, test_labels)
  T = 1

  for predsamples in (range(10)):
    if predsamples == 0 :
      predictions = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      predictions = predictions.reshape((1,) + predictions.shape)
    else:
      pred = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      pred = pred.reshape((1,) + pred.shape)
      predictions = np.concatenate((predictions, pred))

  return predictions

def model_test_eval(model, test_images, test_labels):
  T = 1
  pred = np.array(tf.nn.softmax(model.predict(test_images)/T))
  for i in range(1,10):
    pred += np.array(tf.nn.softmax(model.predict(test_images)))
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  acc = sum(acc)/len(acc)
  print("test set 성능 : " + str(acc))

In [ ]:
def basic_augmentation(imagearray):
  image = Image.fromarray(imagearray)
  tr1 = transforms.RandomHorizontalFlip()
  tr2 = transforms.RandomRotation(10)
  tr3 = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)
  tr4 = transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2))
  image = tr1(tr2(tr3(tr4(image))))
  return(np.array(image))

def makeaugs(n, input):
  augs = []
  for j in range(n):
    for i in input:
      augs.append(basic_augmentation(np.array(i, np.uint8)))
  return(np.array(augs))

In [ ]:
def sample_beta_distribution(size, concentration_0=0.3, concentration_1=0.3):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

def mixup (size, data, alpha = 0.2):
  image, label = data
  L = sample_beta_distribution(size, alpha, alpha)
  XL = tf.reshape(L, (size, 1, 1, 1))
  YL = tf.reshape(L, (size, 1))
  IND1 = np.random.choice(len(label), size)
  IND2 = np.random.choice(len(label), size)
  newimage = XL*image[IND1] + (1-XL)*image[IND2]
  newlabel = YL*label[IND1] + (1-YL)*label[IND2]
  return (newimage, newlabel)

In [ ]:
# model = cnn_13()
# X = lbl_train_images
# y = lbl_train_labels

# X = np.concatenate([makeaugs(10, X), X])
# y = np.concatenate([y,y,y,y,y,y,y,y,y,y, y])

# predictions = fit_and_labeling_cnn_13(200, 64)

In [ ]:
# schedule = list(2**np.array(list(range(5)))/(2**5))
# schedule = [[i,i,i,i,i] for i in schedule]
# schedule = np.array(schedule).reshape([-1])

# alpha = 0.2
# first = time.time()

# for iter in range(len(schedule)):
#     pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
#     conf = np.max(np.mean(predictions, axis=0), axis=1)
#     uncert = np.std(predictions, axis=0)
#     uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])
#     cert = 1-uncert
    
#     ubl_pseudo_labels = []
#     for i in pseudo:
#         temp = [0,0,0,0,0,0,0,0,0,0]
#         temp[i] = 1
#         ubl_pseudo_labels.append(temp)
#     ubl_pseudo_labels = np.array(ubl_pseudo_labels)

#     score = alpha*conf + (1-alpha)*cert
#     score = (score-min(score))/(max(score)-min(score))+0.0001
#     score = np.exp(score/schedule[iter])
#     score = score/sum(score)

#     indx = np.random.choice(len(score), 50000, p = score)

#     X = ubl_train_images[indx]
#     y = ubl_pseudo_labels[indx]

#     X = np.concatenate([lbl_train_images, X])
#     y = np.concatenate([lbl_train_labels, y])

#     size = len(y) * 4
#     newimage, newlabel = mixup(size, (X, y))
#     augimage, auglabel = makeaugs(4, X), np.concatenate((y,y,y,y))
#     X = np.concatenate((newimage, augimage))
#     y = np.concatenate((newlabel, auglabel))

#     print("< iter "+str(iter)+" evaluation >")
#     predictions = fit_and_labeling_cnn_13(25, 64)
#     print(time.time() - first)
    
#     del newimage, newlabel, augimage, auglabel, X, y
#     gc.collect()

In [ ]:
for alphas in [0, 0.2, 0.4, 0.6, 0.8, 1.0]:
    
    print("<<< alpha "+str(alphas)+" evaluation >>>")
    
    model = cnn_13()
    X = lbl_train_images
    y = lbl_train_labels

    X = np.concatenate([makeaugs(10, X), X])
    y = np.concatenate([y,y,y,y,y,y,y,y,y,y, y])

    predictions = fit_and_labeling_cnn_13(200, 64)

#     schedule = list(2**np.array(list(range(5)))/(2**5))
    schedule = list(2**np.array(list(range(2)))/(2**5))
    schedule = [[i,i,i,i,i] for i in schedule]
    schedule = np.array(schedule).reshape([-1])

    alpha = alphas
    first = time.time()

    for iter in range(len(schedule)):
        pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
        conf = np.max(np.mean(predictions, axis=0), axis=1)
        uncert = np.std(predictions, axis=0)
        uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])
        cert = 1-uncert

        ubl_pseudo_labels = []
        for i in pseudo:
            temp = [0,0,0,0,0,0,0,0,0,0]
            temp[i] = 1
            ubl_pseudo_labels.append(temp)
        ubl_pseudo_labels = np.array(ubl_pseudo_labels)

        score = alpha*conf + (1-alpha)*cert
        score = (score-min(score))/(max(score)-min(score))+0.0001
        score = np.exp(score/schedule[iter])
        score = score/sum(score)

        indx = np.random.choice(len(score), 50000, p = score)

        X = ubl_train_images[indx]
        y = ubl_pseudo_labels[indx]

        X = np.concatenate([lbl_train_images, X])
        y = np.concatenate([lbl_train_labels, y])

        size = len(y) * 4
        newimage, newlabel = mixup(size, (X, y))
        augimage, auglabel = makeaugs(4, X), np.concatenate((y,y,y,y))
        X = np.concatenate((newimage, augimage))
        y = np.concatenate((newlabel, auglabel))

        print("< iter "+str(iter)+" evaluation >")
        predictions = fit_and_labeling_cnn_13(25, 64)
        print(time.time() - first)

        del newimage, newlabel, augimage, auglabel, X, y
        gc.collect()

<<< alpha 0 evaluation >>>
test set 성능 : 0.5987
< iter 0 evaluation >
test set 성능 : 0.6687
4783.803838014603
< iter 1 evaluation >
test set 성능 : 0.6809
9549.411909103394
< iter 2 evaluation >
test set 성능 : 0.6853
14341.644701242447
< iter 3 evaluation >
test set 성능 : 0.6996
19135.61491894722
< iter 4 evaluation >
test set 성능 : 0.7042
23910.841858148575
< iter 5 evaluation >
test set 성능 : 0.7079
28718.722818374634
< iter 6 evaluation >
test set 성능 : 0.7183
33548.474012851715
< iter 7 evaluation >
test set 성능 : 0.726
38351.46155834198
< iter 8 evaluation >
test set 성능 : 0.7232
43181.923592567444
< iter 9 evaluation >
test set 성능 : 0.7318
48032.58542919159
<<< alpha 0.2 evaluation >>>
test set 성능 : 0.6129
< iter 0 evaluation >
test set 성능 : 0.6598
4829.364767074585
< iter 1 evaluation >
test set 성능 : 0.6783
9665.686887741089
< iter 2 evaluation >
test set 성능 : 0.6917
14522.985300064087
< iter 3 evaluation >
test set 성능 : 0.697
19379.943120479584
< iter 4 evaluation >
test set 성능 : 0.7059
